In [1]:
from dotenv import load_dotenv
load_dotenv('../wawr_ingestion.env')
from ekb.base.sql_interface import Session
import pandas as pd

In [2]:
nodes = pd.read_sql(sql="select * from elements where record_type='node'", con=Session().get_bind())

In [3]:
nodes

,record_type,id,type_id,meta,text,created_date,date,status
0,node,2109.02707,PaperAbstract,"{'submitter': 'Xueqing Wu', 'authors': 'Xueqin...",We study a new problem setting of informatio...,2024-02-04 21:10:39.869479,2021-09-06 19:35:46,
1,node,2109.02772,PaperAbstract,"{'submitter': 'Tianxing He', 'authors': 'Tianx...",Prompt-based knowledge probing for 1-hop rel...,2024-02-04 21:10:39.869479,2021-09-06 23:29:36,
2,node,2109.02837,PaperAbstract,"{'submitter': 'Kaixin Ma', 'authors': 'Kaixin ...",Commonsense reasoning benchmarks have been l...,2024-02-04 21:10:39.869980,2021-09-07 03:13:06,
3,node,2109.03009,PaperAbstract,"{'submitter': 'Haiqin Yang', 'authors': 'Mengy...","Recently, large pre-trained neural language ...",2024-02-04 21:10:39.869980,2021-09-07 11:48:23,
4,node,2109.03529,PaperAbstract,"{'submitter': 'Yekun Chai', 'authors': 'Yekun ...",Automatically translating images to texts in...,2024-02-04 21:10:39.870480,2021-09-08 10:12:14,
...,...,...,...,...,...,...,...,...
68857,node,2109.02550,PaperAbstract,"{'submitter': 'Kris Cao', 'authors': 'Kris Cao...",Neural language models typically tokenise in...,2024-02-04 21:10:39.868980,2021-09-06 15:37:02,
68858,node,2109.02555,PaperAbstract,"{'submitter': 'Milad Moradi', 'authors': 'Mila...",Deep neural language models have set new bre...,2024-02-04 21:10:39.869479,2021-09-06 15:50:37,
68859,node,2109.02572,PaperAbstract,"{'submitter': 'Wanyun Cui', 'authors': 'Wanyun...",We study how to enhance text representation ...,2024-02-04 21:10:39.869479,2021-09-06 16:16:10,
68860,node,2109.02593,PaperAbstract,"{'submitter': 'Peter Clark', 'authors': 'Oyvin...",Despite the successes of pretrained language...,2024-02-04 21:10:39.869479,2021-09-06 16:37:46,


In [4]:
meta_series = nodes['meta'].apply(pd.Series)

C:\Users\Mihai\AppData\Local\Temp\ipykernel_11208\1717915931.py:1: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  meta_series = nodes['meta'].apply(pd.Series)


In [5]:
meta_series.columns

Index(['submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi',
       'report-no', 'categories', 'license', 'versions', 'update_date',
       'authors_parsed', 'lm'],
      dtype='object')

In [6]:
nodes.columns

Index(['record_type', 'id', 'type_id', 'meta', 'text', 'created_date', 'date',
       'status'],
      dtype='object')

In [38]:
nodes.loc[nodes["type_id"]=="GraphNode", "type_id"]="Knowledge"
nodes.loc[(nodes["id"].str.startswith("entity")), "type_id"]="Entity"


In [39]:
nodes[nodes["type_id"]=="Entity"][:10]

,record_type,id,type_id,meta,text,created_date,date,status
30,node,entity:audio hallucination classification,Entity,{},audio hallucination classification,2024-02-11 13:22:48.844188,NaT,
41,node,entity:GPT-3,Entity,{},GPT-3,2024-02-11 14:42:32.057781,NaT,
67,node,entity:pre-trained audio-text models,Entity,{},pre-trained audio-text models,2024-02-11 13:22:48.844688,NaT,
98,node,entity:MIA experiments,Entity,{},MIA experiments,2024-02-11 13:20:00.303709,NaT,
99,node,entity_type:project component,Entity,{},project component,2024-02-11 13:20:00.303709,NaT,
100,node,entity:Embedding Attack Project,Entity,{},Embedding Attack Project,2024-02-11 13:20:00.303709,NaT,
120,node,entity:MIA strategies,Entity,{},MIA strategies,2024-02-11 13:20:00.304209,NaT,
122,node,entity:AI models,Entity,{},AI models,2024-02-11 14:43:08.640281,NaT,
123,node,entity:model's vulnerability,Entity,{},model's vulnerability,2024-02-11 13:20:00.305209,NaT,
124,node,entity_type:subject,Entity,{},subject,2024-02-11 14:43:40.917781,NaT,


In [41]:
cols = ["id", "type_id", "text", "created_date", "date", "status", "title"]

In [42]:
nodes_save=pd.concat((nodes, meta_series), axis=1)[cols]
for col in ["text", "title", "status"]:
    nodes_save[col] = nodes_save[col].str.replace("\"", "'")
nodes_save.to_csv("nodes.csv", index=False)

In [43]:
nodes_save.shape

(68862, 7)

In [26]:
relationships = pd.read_sql(sql="SELECT * FROM public.relationships a left join elements b on a.id=b.id", con=Session().get_bind())


In [51]:
cols = ["id", "from_node_id", "to_node_id", "text", "created_date", "date", "status"]
for col in ["text", "from_node_id", "to_node_id"]:
    relationships[col] = relationships[col].str.replace("\"", "'")
relationships[cols].to_csv('relationships.csv', index=False)

In [44]:
relationships.iloc[:10]

,id,from_node_id,to_node_id,record_type,id,type_id,meta,text,created_date,date,status
0,00003994-128e-46db-bd5b-d7a632e849e5,54ae237c-155d-49fc-be1c-18446a5c3131,fact_type:result,relationship,00003994-128e-46db-bd5b-d7a632e849e5,GraphRelationship,{},is a,2024-02-11 14:02:34.064690,None,
1,000076b5-bf54-449b-8fa5-5bf5338ef691,entity:large branching factors,29ddd9ab-105a-43d0-b176-94f50066bdd7,relationship,000076b5-bf54-449b-8fa5-5bf5338ef691,GraphRelationship,{},mentioned in,2024-02-11 14:04:43.250000,None,
2,0000b7d9-f72c-43a8-a1a4-649805314d96,ad78aa72-cf4d-4d2b-b826-5e2f291e4dfd,2312.00849,relationship,0000b7d9-f72c-43a8-a1a4-649805314d96,GraphRelationship,{},mentioned in,2024-02-11 14:09:45.259433,None,
3,0001b4ec-7a57-4867-908b-a644458100d3,entity:state-of-the-art defense methods,6e01b3c1-f586-4b0b-9dec-062ab49edbc4,relationship,0001b4ec-7a57-4867-908b-a644458100d3,GraphRelationship,{},mentioned in,2024-02-11 13:58:08.569733,None,
4,0001e4b9-b8cd-4bb8-bebf-6f6d76160995,entity:Reinforcement learning,0841d6e5-ea2f-4d2a-af74-2b12a0ce21b0,relationship,0001e4b9-b8cd-4bb8-bebf-6f6d76160995,GraphRelationship,{},mentioned in,2024-02-11 14:03:23.926423,None,
5,00025fc7-6ceb-4f55-81ab-fdaba722f622,entity:high-quality demonstration samples,a3663f93-6ee2-4307-bbc5-f898816bc7ab,relationship,00025fc7-6ceb-4f55-81ab-fdaba722f622,GraphRelationship,{},mentioned in,2024-02-11 14:07:59.496284,None,
6,0002c9e7-809d-4c5b-87a4-f1e078ae3a0c,entity:backbone language models,03df95f0-1d15-46be-9b81-b1af7a8da2ad,relationship,0002c9e7-809d-4c5b-87a4-f1e078ae3a0c,GraphRelationship,{},mentioned in,2024-02-11 14:34:38.067781,None,
7,0002ceed-b492-4374-8204-e5c0516d6aa0,entity:external human confidence,730601ce-c673-490f-9747-ab92b8735744,relationship,0002ceed-b492-4374-8204-e5c0516d6aa0,GraphRelationship,{},mentioned in,2024-02-11 13:20:11.303055,None,
8,0003431b-4eb4-4802-87ad-0d77c4aae191,entity:Myers-Briggs Type Indicator (MBTI),entity_type:test,relationship,0003431b-4eb4-4802-87ad-0d77c4aae191,GraphRelationship,{},is a,2024-02-11 13:36:39.458641,None,
9,0003d5db-1227-4b22-8bf9-6289fa5c57ad,entity:Pretraining monolingual language models,entity_type:model,relationship,0003d5db-1227-4b22-8bf9-6289fa5c57ad,GraphRelationship,{},is a,2024-02-11 13:26:35.688672,None,


In [46]:
relationships.shape

(135660, 11)

In [47]:
relationships[relationships["from_node_id"].str.contains("\"")]


,id,from_node_id,to_node_id,record_type,id,type_id,meta,text,created_date,date,status
5618,0aa611d2-7b5b-418f-a8d1-f2fc284afe9b,"entity:factually incorrect responses or ""hallu...",0746670b-2ee0-4216-930c-cf119e21aaa9,relationship,0aa611d2-7b5b-418f-a8d1-f2fc284afe9b,GraphRelationship,{},mentioned in,2024-02-11 14:13:30.200965,None,
6204,0baca37c-b3b1-4015-9f66-bba83c8819a6,"entity:""do not generate abusive content""",entity_type:rule,relationship,0baca37c-b3b1-4015-9f66-bba83c8819a6,GraphRelationship,{},is a,2024-02-11 14:39:19.429281,None,
9410,1197242b-aa1c-4ffc-bde3-57e6b114d06b,"entity:features of ""low-quality"" code",e7e7c3b8-cbfe-4ef6-96e7-8e186ba471a9,relationship,1197242b-aa1c-4ffc-bde3-57e6b114d06b,GraphRelationship,{},mentioned in,2024-02-11 14:08:31.408719,None,
9944,1295779b-3208-4dae-9fcc-0711d04b4244,"entity:""pick and place""",entity_type:task,relationship,1295779b-3208-4dae-9fcc-0711d04b4244,GraphRelationship,{},is a,2024-02-11 13:35:16.267208,None,
10904,145e06e6-c85b-4d01-bec0-edf6243c804f,"entity:T\""ULU 2+DPO 70B",entity_type:model,relationship,145e06e6-c85b-4d01-bec0-edf6243c804f,GraphRelationship,{},is a,2024-02-11 14:31:03.442290,None,
12575,17904538-c922-4b98-b86e-1708375e0c00,"entity:""in-the-wild"" data",entity_type:dataset,relationship,17904538-c922-4b98-b86e-1708375e0c00,GraphRelationship,{},is a,2024-02-11 13:24:02.509468,None,
16787,1f89cb7d-ae77-4628-b338-34c3c30384d4,"entity:T\""ULU",78bfe4f8-7d92-4728-9313-931bf57487b8,relationship,1f89cb7d-ae77-4628-b338-34c3c30384d4,GraphRelationship,{},mentioned in,2024-02-11 14:31:03.439790,None,
17904,21a33e67-79e8-460c-ac49-6cf0b473352f,"entity:pipelines of safety techniques (""protoc...",4f2b19a4-3afd-4778-9c9e-0b13c06c134b,relationship,21a33e67-79e8-460c-ac49-6cf0b473352f,GraphRelationship,{},mentioned in,2024-02-11 14:04:45.557151,None,
18890,239d0ca0-1eda-4368-b557-923577e1ddc2,"entity:annotation ""blind spots""",entity_type:issue,relationship,239d0ca0-1eda-4368-b557-923577e1ddc2,GraphRelationship,{},is a,2024-02-11 14:10:37.631171,None,
19246,244e7b91-899c-4240-b88e-a1ce0550b0af,"entity:""Bob""",4c436121-434f-441a-a3e6-44cb4aa68454,relationship,244e7b91-899c-4240-b88e-a1ce0550b0af,GraphRelationship,{},mentioned in,2024-02-11 14:11:22.992692,None,


In [49]:
nodes_save[nodes_save["id"].str.contains("'")]

,id,type_id,text,created_date,date,status,title
123,entity:model's vulnerability,Entity,model's vulnerability,2024-02-11 13:20:00.305209,NaT,,NaN
307,entity:LLM's internal confidence,Entity,LLM's internal confidence,2024-02-11 13:20:11.304555,NaT,,NaN
309,entity:model's actual confidence levels,Entity,model's actual confidence levels,2024-02-11 13:20:11.304555,NaT,,NaN
1552,entity:baseline's trend,Entity,baseline's trend,2024-02-11 13:22:00.873029,NaT,,NaN
4476,entity:model's response,Entity,model's response,2024-02-11 13:22:20.957437,NaT,,NaN
...,...,...,...,...,...,...,...
64434,entity:LLaMA's state-of-the-art performance,Entity,LLaMA's state-of-the-art performance,2024-02-11 14:43:48.791781,NaT,,NaN
64477,entity:seven unique facets of GPT-4V's behavior,Entity,seven unique facets of GPT-4V's behavior,2024-02-11 14:44:00.149281,NaT,,NaN
66549,entity:system's superiority,Entity,system's superiority,2024-02-11 13:24:21.819805,NaT,,NaN
68710,entity:student model's performance,Entity,student model's performance,2024-02-11 13:19:46.987343,NaT,,NaN
